# Backborn 
YOLO V3는 백본으로 DarkNet-53을 사용한다.

In [1]:
# 필요한 라이브러리
import torch
from torch import nn


In [20]:
# 입력이미지 랜덤생성
input_image=torch.randn(1,3,256,256)
print(input_image.shape)

torch.Size([1, 3, 256, 256])


In [48]:
def DBL(in_c,out_c,kernel_size,stride,padding):
    dbl_block=nn.Sequential(nn.Conv2d(in_c,out_c,kernel_size=kernel_size,padding=padding,stride=stride),
                            nn.BatchNorm2d(out_c),
                            nn.LeakyReLU())
    return dbl_block

class Res_unit(nn.Module):
    def __init__(self,in_c):
        super(Res_unit,self).__init__()
        
        reduce_c=int(in_c/2)
        self.layer1=DBL(in_c,reduce_c,1,1,0)
        self.layer2=DBL(reduce_c,in_c,3,1,1)
    
    def forward(self,x):
        res_connection= x
        out=self.layer1(x)
        out=self.layer2(out)
        out=out+res_connection
        return out
    

In [64]:
class Darknet53(nn.Module):
    def __init__(self,block):
        super(Darknet53,self).__init__()
        
        self.conv1=DBL(3,32,3,1,1)
        self.conv2=DBL(32,64,3,2,1)
        
        self.res_block1=self.num_block(block,64,num=1)
        self.conv3=DBL(64,128,3,2,1)
        
        self.res_block2=self.num_block(block,128,2)
        self.conv4=DBL(128,256,3,2,1)
        
        self.res_block3=self.num_block(block,256,8)
        self.conv5=DBL(256,512,3,2,1) # 3*3 conv하면 마진1 > 패딩으로 채워줌
        
        self.res_block4=self.num_block(block,512,8)
        self.conv6=DBL(512,1024,3,2,1)
        
        self.res_block5=self.num_block(block,1024,4)

    def forward(self,x):
        x=self.conv1(x)
        print(x.shape)
        x=self.conv2(x)
        print(x.shape)
        x=self.res_block1(x)
        print(x.shape)
        x=self.conv3(x)
        print(x.shape)
        x=self.res_block2(x)
        print(x.shape)
        x=self.conv4(x)
        print(x.shape)
        x=self.res_block3(x)
        feature1=x
        print(x.shape)
        x=self.conv5(x)
        print(x.shape)
        x=self.res_block4(x)
        feature2=x
        print(x.shape)
        x=self.conv6(x)
        print(x.shape)
        x=self.res_block5(x)
        print(x.shape)
        
        return feature1,feature2
    def num_block(self,block,in_c,num):
        layers=[]
        for i in range(num):
            layers.append(Res_unit(in_c))
        return nn.Sequential(*layers)
        

In [65]:
Darknet53(Res_unit).forward(input_image)

torch.Size([1, 32, 256, 256])
torch.Size([1, 64, 128, 128])
torch.Size([1, 64, 128, 128])
torch.Size([1, 128, 64, 64])
torch.Size([1, 128, 64, 64])
torch.Size([1, 256, 32, 32])
torch.Size([1, 256, 32, 32])
torch.Size([1, 512, 16, 16])
torch.Size([1, 512, 16, 16])
torch.Size([1, 1024, 8, 8])
torch.Size([1, 1024, 8, 8])


(tensor([[[[4.6661, 1.2224, 2.4411,  ..., 5.7521, 1.2889, 3.9169],
           [5.3882, 1.4670, 2.3789,  ..., 4.0531, 1.8666, 3.3979],
           [0.5329, 3.6149, 6.6258,  ..., 6.3828, 4.0576, 1.5868],
           ...,
           [1.0965, 2.5256, 2.5499,  ..., 2.5864, 1.5584, 4.7937],
           [1.2321, 2.3293, 3.1193,  ..., 4.4179, 4.7045, 3.3719],
           [3.1619, 2.3882, 2.4734,  ..., 8.5350, 3.3591, 2.7660]],
 
          [[2.9688, 2.7631, 2.1813,  ..., 4.4676, 3.5703, 2.8273],
           [1.8153, 2.3276, 3.2053,  ..., 2.9071, 2.8288, 3.8519],
           [1.4332, 2.1110, 2.6932,  ..., 1.2331, 3.6366, 4.4504],
           ...,
           [2.7583, 1.5076, 4.0471,  ..., 2.2275, 2.7969, 4.4192],
           [2.8280, 2.5737, 1.5420,  ..., 4.6525, 5.6351, 4.2080],
           [2.8247, 4.4701, 0.8411,  ..., 2.1215, 3.3373, 5.6340]],
 
          [[2.0306, 3.9819, 5.8432,  ..., 5.5235, 3.7163, 3.1650],
           [2.8207, 2.8727, 1.7922,  ..., 2.1839, 5.1793, 5.0265],
           [1.7218, 1.94

In [ ]:
class Yolo_v3(nn.Module):
    def __init__(self):
        